In [1]:
import xarray as xr


url = "https://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/2017_da/avg/Averages_Best"
ds = xr.open_dataset(url)

In [2]:
time_slice = {"time": "2022-06-06"}
surface = {"s_rho": -1}

ds = ds[["temp", "salt"]].sel(time_slice).isel(surface)

In [3]:
import zarr


compressor = zarr.Blosc(clevel=2, shuffle=-1)


fname = "doppio/doppio_compressed.zarr"

ds.to_zarr(
    fname,
    mode="w",
    safe_chunks=True,
    consolidated=True,
    encoding={
        var: {"compressor": compressor} for var in ds.variables
    },
);

In [4]:
!ls doppio/*zarr
!du -h doppio/*zarr

lat_rho  lon_rho  salt	s_rho  temp  time  time_run
144K	doppio/doppio_compressed.zarr/salt
156K	doppio/doppio_compressed.zarr/lat_rho
148K	doppio/doppio_compressed.zarr/temp
16K	doppio/doppio_compressed.zarr/time
16K	doppio/doppio_compressed.zarr/s_rho
16K	doppio/doppio_compressed.zarr/time_run
156K	doppio/doppio_compressed.zarr/lon_rho
676K	doppio/doppio_compressed.zarr


In [5]:
subset = xr.open_zarr(fname)

subset

<xarray.Dataset>
Dimensions:   (eta_rho: 106, xi_rho: 242, time: 1)
Coordinates:
    lat_rho   (eta_rho, xi_rho) float64 dask.array<chunksize=(106, 242), meta=np.ndarray>
    lon_rho   (eta_rho, xi_rho) float64 dask.array<chunksize=(106, 242), meta=np.ndarray>
    s_rho     float64 ...
  * time      (time) datetime64[ns] 2022-06-06T12:00:00
    time_run  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    salt      (time, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 106, 242), meta=np.ndarray>
    temp      (time, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 106, 242), meta=np.ndarray>
Attributes: (12/46)
    CPP_options:                     DOPPIO, ADD_FSOBC, ADD_M2OBC, ANA_BSFLUX...
    Conventions:                     CF-1.4, SGRID-0.3
    DODS_EXTRA.Unlimited_Dimension:  ocean_time
    NLM_LBC:                         \nEDGE:  WEST   SOUTH  EAST   NORTH  \nz...
    NLM_TADV:                        \nADVECTION:   HORIZONTAL   VERTICAL    ...
    _CoordSysBuilder:                ucar.nc2.dataset.conv.CF1Convention
    ...                              ...
    svn_url:                         https://www.myroms.org/svn/src/trunk
    tide_file:                       /home/om/roms/doppio/7km/doppio_tide_7km.nc
    tiling:                          004x004
    title:                           ROMS doppio Real-Time Operational PSAS F...
    type:                            ROMS/TOMS nonlinear model averages file
    var_info:                        ../Data/varinfo1040t_daily.dat

In [8]:
from pathlib import Path

path = Path("doppio").absolute()
fname = f"file://{path}#mode=xarray,file"
print(fname)

file:///home/filipe/IOOS/ioos_code_lab/jupyterbook/content/code_gallery/data_management_notebooks/doppio#mode=xarray,file


# TODO this will fail b/c we need libnetcdf 4.9.0

In [9]:
!ncdump -h "$fname"